# 分析Ajax请求爬取今日头条街拍美图

### 项目流程：
#### 抓取索引页内容：利用requests请求目标站点，得到索引网页HTML代码，返回结果
#### 抓取详情页内容：解析返回结果，得到详情页的链接，并进一步的抓取详情页内容
#### 下载图片与保存数据库：将图片下载到本地，并把页面信息及图片URL保存至MangoDB
#### 开启循环及多线程：对多页内容遍历，开启多线程提高抓取速度
### 技术路径：requests re json BeautifulSoup pymongo
### 根据初始url获取索引页html——>解析索引页html获取url——>抓取url对应html——>解析url对应html

In [15]:
# !/usr/bin/env python3
# -*- coding:utf-8 -*-

import requests
from requests.exceptions import RequestException
from bs4 import BeautifulSoup

import os
import re
import json
import pymongo
from config import *
from hashlib import md5
from multiprocessing import Pool

client = pymongo.MongoClient(MONGO_URL) # 申明一个mongodb对象
db = client[MONGO_DB] # 数据库

def get_index_page(offset, keyword):
    data = {
        'offset': offset,
        'format': 'json',
        'keyword': keyword,
        'autoload': 'true',
        'count': 20,
        'cur_tab': 1,
        'from': 'gallery'}
    url = 'https://www.toutiao.com/search_content/'
    try:
        res = requests.get(url, params=data)
        res.raise_for_status()
        res.encoding = res.apparent_encoding
        return res.text
    except RequestException:
        print('Requests Error')
        return None
    
def parse_index_page(html):
    data = json.loads(html)
    if data and 'data' in data.keys():
        for item in data['data']:
            yield item.get('article_url')
            
def get_url_html(url):
    try:
        res = requests.get(url)
        res.raise_for_status()
        res.encoding = res.apparent_encoding
        return res.text
    except RequestException:
        print('Requests URL Error', url)
        return None
    
def parse_url_detail(html, url):
    soup = BeautifulSoup(html, 'lxml')
    title = soup.select('title')[0].get_text()
    print(title)
    image_pattern = re.compile('gallery: JSON.parse\((.*?)\),\n',re.S)
    result = re.search(image_pattern, html)
    if result:
        data = json.loads(json.loads(result.group(1)))
        if data and 'sub_images' in data.keys():
            sub_images = data.get('sub_images')
            images = [item.get('url') for item in sub_images]
            for image in images:
                download_images(image)
            return {
                'title': title,
                'url': url,
                'images': images
            }

def save_to_mongo(result):
    try:
        if db[MONGO_TABLE].insert(result):
            print('存储到MongoDB成功', result)
            return True
    except:
        pass

def download_images(url):
    print('正在下载', url)
    try:
        res = requests.get(url)
        res.raise_for_status()
        save_images(res.content)
    except RequestException:
        print('请求图片失败', url)
        return None

def save_images(content):
    dir_name = str(GROUP_START)+'-'+str(GROUP_END)#个人补充功能！
    dir_path = 'E:/spiderData/images/jiepai/{}'.format(dir_name)
    file_path = '{0}/{1}.{2}'.format(dir_path, md5(content).hexdigest(),'jpg')
    try:
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
    except:
        print('DIR has been maked')
        pass
    try:
        if not os.path.exists(file_path):
            with open(file_path, 'wb') as f:
                f.write(content)
                f.close()
    except:
        print('图片已存在')
    
    
def main(offset, KEYWORD):
    html = get_index_page(offset, KEYWORD)
    for url in parse_index_page(html):
        html = get_url_html(url)
        if html:
            result = parse_url_detail(html, url)
            if result:
                save_to_mongo(result)
    
if __name__ == '__main__':
    groups = [x * 20 for x in range(GROUP_START, GROUP_END + 1)]
    for group in groups:
        main(group, KEYWORD)
    #pool = Pool()
    #pool.map(main, groups)

街拍优雅美女高跟街拍
正在下载 http://p1.pstatp.com/origin/53ea0004f6a0df547bb2
正在下载 http://p1.pstatp.com/origin/53ea0004f6ac3d6e6d85
正在下载 http://p3.pstatp.com/origin/53eb0004a2429624b71d
正在下载 http://p1.pstatp.com/origin/53ec0002f3f687587628
正在下载 http://p1.pstatp.com/origin/53ed000288f2cc6af474
正在下载 http://p1.pstatp.com/origin/53e4000520d15bc1b5c7
正在下载 http://p3.pstatp.com/origin/53e4000520dfadebeccb
正在下载 http://p1.pstatp.com/origin/53ec0002f41373e4562f


C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:75: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


存储到MongoDB成功 {'title': '街拍优雅美女高跟街拍', 'url': 'http://toutiao.com/group/6504903362155119118/', 'images': ['http://p1.pstatp.com/origin/53ea0004f6a0df547bb2', 'http://p1.pstatp.com/origin/53ea0004f6ac3d6e6d85', 'http://p3.pstatp.com/origin/53eb0004a2429624b71d', 'http://p1.pstatp.com/origin/53ec0002f3f687587628', 'http://p1.pstatp.com/origin/53ed000288f2cc6af474', 'http://p1.pstatp.com/origin/53e4000520d15bc1b5c7', 'http://p3.pstatp.com/origin/53e4000520dfadebeccb', 'http://p1.pstatp.com/origin/53ec0002f41373e4562f'], '_id': ObjectId('5a795d164124bc23549b66a6')}
街拍——南京美女
正在下载 http://p3.pstatp.com/origin/46eb00032b011afb3f9c
正在下载 http://p9.pstatp.com/origin/46eb00032b02c1c6d784
正在下载 http://p1.pstatp.com/origin/46ed0003118d7e273dac
正在下载 http://p3.pstatp.com/origin/46ef0002ccd6abab8a8d
正在下载 http://p1.pstatp.com/origin/46ee0002ebdd82c66e6d
存储到MongoDB成功 {'title': '街拍——南京美女', 'url': 'http://toutiao.com/group/6490040284989096461/', 'images': ['http://p3.pstatp.com/origin/46eb00032b011afb3f9c', '

Gucci 红色系鞋子，要不要这么美嘛
正在下载 http://p3.pstatp.com/origin/615b000074607fb3b450
正在下载 http://p1.pstatp.com/origin/615b00007463e41ead1e
正在下载 http://p1.pstatp.com/origin/615800044f66c4b2934f
正在下载 http://p1.pstatp.com/origin/615800044f6981f6eb8c
正在下载 http://p3.pstatp.com/origin/5e8b00056d265c555dc4
正在下载 http://p3.pstatp.com/origin/615800044f6e0939ccf0
正在下载 http://p1.pstatp.com/origin/615b0000746c2907e8a6
正在下载 http://p3.pstatp.com/origin/61590000b64340a4ebf1
正在下载 http://p3.pstatp.com/origin/615800044f76cd64be7d
存储到MongoDB成功 {'title': 'Gucci 红色系鞋子，要不要这么美嘛', 'url': 'http://toutiao.com/group/6519339566141276675/', 'images': ['http://p3.pstatp.com/origin/615b000074607fb3b450', 'http://p1.pstatp.com/origin/615b00007463e41ead1e', 'http://p1.pstatp.com/origin/615800044f66c4b2934f', 'http://p1.pstatp.com/origin/615800044f6981f6eb8c', 'http://p3.pstatp.com/origin/5e8b00056d265c555dc4', 'http://p3.pstatp.com/origin/615800044f6e0939ccf0', 'http://p1.pstatp.com/origin/615b0000746c2907e8a6', 'http://p3.pstatp

图虫街拍摄影：街拍
正在下载 http://p1.pstatp.com/origin/tuchong.fullscreen/22323399_tt
正在下载 http://p3.pstatp.com/origin/tuchong.fullscreen/22323400_tt
正在下载 http://p1.pstatp.com/origin/tuchong.fullscreen/22323401_tt
正在下载 http://p3.pstatp.com/origin/tuchong.fullscreen/22323402_tt
正在下载 http://p1.pstatp.com/origin/tuchong.fullscreen/22323403_tt
正在下载 http://p3.pstatp.com/origin/tuchong.fullscreen/22323405_tt
正在下载 http://p3.pstatp.com/origin/tuchong.fullscreen/22323406_tt
正在下载 http://p9.pstatp.com/origin/tuchong.fullscreen/22323407_tt
存储到MongoDB成功 {'title': '图虫街拍摄影：街拍', 'url': 'http://toutiao.com/group/6489423487139054093/', 'images': ['http://p1.pstatp.com/origin/tuchong.fullscreen/22323399_tt', 'http://p3.pstatp.com/origin/tuchong.fullscreen/22323400_tt', 'http://p1.pstatp.com/origin/tuchong.fullscreen/22323401_tt', 'http://p3.pstatp.com/origin/tuchong.fullscreen/22323402_tt', 'http://p1.pstatp.com/origin/tuchong.fullscreen/22323403_tt', 'http://p3.pstatp.com/origin/tuchong.fullscreen/22323405_tt', 'ht

...

图虫街拍摄影：昨日街拍
正在下载 http://p1.pstatp.com/origin/101e0001b20dc51022db
正在下载 http://p9.pstatp.com/origin/101e0001b20f6c645e92
正在下载 http://p1.pstatp.com/origin/101f00029ac9b51cf9c0
正在下载 http://p1.pstatp.com/origin/f7c0004eb9cbbb13ec5
正在下载 http://p3.pstatp.com/origin/f7c0004eb9eb8844907
正在下载 http://p9.pstatp.com/origin/101e0001b220ceefdcad
正在下载 http://p1.pstatp.com/origin/f7c0004eba65e0ebdaa
正在下载 http://p1.pstatp.com/origin/f7c0004eba98aec0f0b
正在下载 http://p3.pstatp.com/origin/101f00029ae1390d6099
存储到MongoDB成功 {'title': '图虫街拍摄影：昨日街拍', 'url': 'http://toutiao.com/group/6345610155460313346/', 'images': ['http://p1.pstatp.com/origin/101e0001b20dc51022db', 'http://p9.pstatp.com/origin/101e0001b20f6c645e92', 'http://p1.pstatp.com/origin/101f00029ac9b51cf9c0', 'http://p1.pstatp.com/origin/f7c0004eb9cbbb13ec5', 'http://p3.pstatp.com/origin/f7c0004eb9eb8844907', 'http://p9.pstatp.com/origin/101e0001b220ceefdcad', 'http://p1.pstatp.com/origin/f7c0004eba65e0ebdaa', 'http://p1.pstatp.com/origin/f7c0004eba9

正在下载 http://p1.pstatp.com/origin/1af7000568dea53f940f
正在下载 http://p3.pstatp.com/origin/1af7000568df0db76e50
存储到MongoDB成功 {'title': '图虫街拍摄影：手机街拍', 'url': 'http://toutiao.com/group/6310677729956151553/', 'images': ['http://p9.pstatp.com/origin/1b7500022755d5f30bd3', 'http://p3.pstatp.com/origin/1b7600022c581f2f70e9', 'http://p3.pstatp.com/origin/1b7600022c59c32f5463', 'http://p3.pstatp.com/origin/1af3000532fdf88b9197', 'http://p1.pstatp.com/origin/1af7000568dea53f940f', 'http://p3.pstatp.com/origin/1af7000568df0db76e50'], '_id': ObjectId('5a795f864124bc23549b6716')}
图虫街拍摄影：手机街拍
正在下载 http://p3.pstatp.com/origin/1b7400000321279fcfcd
正在下载 http://p3.pstatp.com/origin/1b75000002fa8ccb6bf0
正在下载 http://p3.pstatp.com/origin/1af300030d1a89fa4769
正在下载 http://p1.pstatp.com/origin/1b74000003242955d80f
正在下载 http://p3.pstatp.com/origin/1b7700000301f5401a84
正在下载 http://p3.pstatp.com/origin/1af80001a6829fcee147
正在下载 http://p3.pstatp.com/origin/1af80001a683f9c13722
正在下载 http://p3.pstatp.com/origin/1b7500

图虫街拍摄影：街拍
正在下载 http://p1.pstatp.com/origin/tuchong.fullscreen/21337635_tt
正在下载 http://p1.pstatp.com/origin/tuchong.fullscreen/21337634_tt
正在下载 http://p3.pstatp.com/origin/tuchong.fullscreen/21337637_tt
正在下载 http://p9.pstatp.com/origin/tuchong.fullscreen/21337636_tt
正在下载 http://p3.pstatp.com/origin/tuchong.fullscreen/21337640_tt
正在下载 http://p1.pstatp.com/origin/tuchong.fullscreen/21337639_tt
存储到MongoDB成功 {'title': '图虫街拍摄影：街拍', 'url': 'http://toutiao.com/group/6501044455217299981/', 'images': ['http://p1.pstatp.com/origin/tuchong.fullscreen/21337635_tt', 'http://p1.pstatp.com/origin/tuchong.fullscreen/21337634_tt', 'http://p3.pstatp.com/origin/tuchong.fullscreen/21337637_tt', 'http://p9.pstatp.com/origin/tuchong.fullscreen/21337636_tt', 'http://p3.pstatp.com/origin/tuchong.fullscreen/21337640_tt', 'http://p1.pstatp.com/origin/tuchong.fullscreen/21337639_tt'], '_id': ObjectId('5a795fb54124bc23549b671e')}
图虫街拍摄影：河南街拍
正在下载 http://p3.pstatp.com/origin/tuchong.fullscreen/20757096_tt
正在下载 http

正在下载 http://p9.pstatp.com/origin/tuchong.fullscreen/23642010_tt
正在下载 http://p3.pstatp.com/origin/tuchong.fullscreen/23642150_tt
正在下载 http://p3.pstatp.com/origin/tuchong.fullscreen/23615415_tt
正在下载 http://p3.pstatp.com/origin/tuchong.fullscreen/23642144_tt
正在下载 http://p3.pstatp.com/origin/tuchong.fullscreen/23639404_tt
正在下载 http://p3.pstatp.com/origin/tuchong.fullscreen/23642143_tt
正在下载 http://p3.pstatp.com/origin/tuchong.fullscreen/23642145_tt
正在下载 http://p1.pstatp.com/origin/tuchong.fullscreen/23642146_tt
正在下载 http://p3.pstatp.com/origin/tuchong.fullscreen/23639408_tt
正在下载 http://p3.pstatp.com/origin/tuchong.fullscreen/23642147_tt
正在下载 http://p3.pstatp.com/origin/tuchong.fullscreen/23642148_tt
正在下载 http://p3.pstatp.com/origin/tuchong.fullscreen/23642149_tt
正在下载 http://p3.pstatp.com/origin/tuchong.fullscreen/23642151_tt
正在下载 http://p1.pstatp.com/origin/tuchong.fullscreen/23642152_tt
正在下载 http://p3.pstatp.com/origin/tuchong.fullscreen/23642158_tt
正在下载 http://p3.pstatp.com/origin/tuchong

正在下载 http://p1.pstatp.com/origin/1c5c000145115181153f
存储到MongoDB成功 {'title': '漩涡街拍摄影：巴黎时装周街拍', 'url': 'http://toutiao.com/group/6409786379618451713/', 'images': ['http://p9.pstatp.com/origin/1b8600000599c1e88e81', 'http://p3.pstatp.com/origin/1c5c000145008693cc72', 'http://p1.pstatp.com/origin/1c6000000e08c381fbb8', 'http://p1.pstatp.com/origin/1b830002a3fe062f31cb', 'http://p1.pstatp.com/origin/1b86000005a3e81bbed1', 'http://p3.pstatp.com/origin/1b830002a3f060198040', 'http://p3.pstatp.com/origin/1c5c000144e0f475a5b0', 'http://p1.pstatp.com/origin/1c5d000006180f546a4a', 'http://p1.pstatp.com/origin/1c5d000006083c5789f1', 'http://p1.pstatp.com/origin/1c5c000144e85c3fffb0', 'http://p3.pstatp.com/origin/1c5f00002a63c362a18a', 'http://p1.pstatp.com/origin/1c5c000145115181153f'], '_id': ObjectId('5a795fef4124bc23549b6728')}
图虫街拍摄影：街拍
正在下载 http://p3.pstatp.com/origin/tuchong.fullscreen/24679699_tt
正在下载 http://p1.pstatp.com/origin/tuchong.fullscreen/24679696_tt
正在下载 http://p1.pstatp.com/orig